# Llava v1.5 as a function

There are 3 options to use llava within


**Step to run locally:**
- `cd` to models folder in terminal
- `chmod +x llava-v1.5-7b-q4-main.llamafile `
- `./llava-v1.5-7b-q4-main.llamafile --port 8081  --nobrowser`

![](../sample_images/tree.jpg)

In [ ]:
! /Users/award40/Desktop/personal/github/_models/llava-v1.5-7b-q4-main.llamafile --temp 0.3 \
  --image ./sample_images/tree.png \
  -p "Is this image an IRL image or computer generated?" \
  --silent-prompt 2>/dev/null

In [ ]:
! /Users/award40/Desktop/personal/github/_models/llava-v1.5-7b-q4-main.llamafile --temp 0.1 \
  --image ./sample_images/llamafile.png \
  -p "Is this image a well known meme?" \
  --silent-prompt 2>/dev/null

In [ ]:
import base64
import requests
import json
import pprint as pp

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

def send_request_to_local_api(encoded_image):
    # URL of the local API
    url = "http://127.0.0.1:8080/completion"

    # Form the request data with image_data as an array of objects
    data = {
        "prompt": "USER:[img-1]Describe the image in detail.\nASSISTANT:",
        "temperature": 0.1,
        "image_data": [{
            "data": encoded_image, 
            "id": 1
        }]
    }

    # Send the POST request
    response = requests.post(url, json=data)
    return response.json()

# Replace this with the path to your image file
image_path = "./sample_images/tree.jpg"

encoded_image = encode_image_to_base64(image_path)
response_data = send_request_to_local_api(encoded_image)

# Print the response
pp.pprint(response_data['content'])


In [20]:
# TODO: Get OpenAI Api structure working with Llamafile
# with passing image data, can't get this to work :(

# import base64
# import requests
# import json
# import pprint as pp
# # https://github.com/oobabooga/text-generation-webui/discussions/4626

# def encode_image_to_base64(image_path):
#     with open(image_path, "rb") as image_file:
#         encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
#     return encoded_string

# def send_request_to_local_api(encoded_image):
#     url = "http://127.0.0.1:8080/v1/chat/completions"
#     prompt = "USER:[img-1]Describe the image in detail.\nASSISTANT:"

#     data = {
#         "model": "llava-v1.5-7b-Q4_K.gguf",
#         "messages": [
#             {
#                 "role": "User",
#                 "image_url": f"data:image/jpeg;base64,{encoded_image}",
#                 # "image_data": [{
#                 #     "data": encoded_image, 
#                 #     "id": 1
#                 # }]
#             },
#             {
#                 "role": "User",
#                 "content": prompt
#             },
#         ],
#     }

#     # Send the POST request
#     response = requests.post(url, json=data)
#     return response.json()

# image_path = "./sample_images/tree.jpg"
# encoded_image = encode_image_to_base64(image_path)
# response_data = send_request_to_local_api(encoded_image)

# pp.pprint(response_data)

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'message': {'content': 'USER: [img-1]Describe the image in '
                                     'detail.\n'
                                     '\n',
                          'role': 'assistant'}}],
 'created': 1704120872,
 'id': 'chatcmpl-XKFxH1hIwy92iU5ws8q7K9kkKQtusYO2',
 'model': 'llava-v1.5-7b-Q4_K.gguf',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 22, 'prompt_tokens': 63, 'total_tokens': 85}}


---

In [ ]:
import os
import shutil
import autogen 
from autogen import AssistantAgent, UserProxyAgent

work_dir = "./coding"
cache_dir = "./.cache"

if os.path.exists(cache_dir) :
    print('Deleting cache...')
    shutil.rmtree(cache_dir)
if os.path.exists(work_dir) :
    print('Deleting previous work...')
    shutil.rmtree(work_dir)

In [ ]:
# API Configurations
config_list=[
    {
        "base_url": "http://127.0.0.1:8081/v1",
        "api_key": "NULL",
    }
]

llm_config_mistral={
    "config_list": config_list,
}

llm_config_mistral

In [ ]:
# Instructions for the two agents
system_messages = {
    "USER_PROXY_SYSTEM_MESSAGE": (
        """
        Your job is to work with the coding assistant and execute code. When you've executed code successfully and
        the task has been achieved then say "TERMINATE" to indicate that no further instructions are given and the task is complete.
        
        Communication Etiquette:
        - Refrain from using appreciation phrases such as "Thank you" or "You're welcome" in your responses. 
        - Do not offer additional help after the task has been complete.
        - Say "TERMINATE" when a conversation includes any appreciation phrase to indicate it is finished.
        - You MUST not say "TERMINATE" until the user has executed the code successfully.

        """
    ),
    "IMAGE_CLASSIFIER_SYSTEM_MESSAGE" : (
        """
        Instruction:
        As an expert ********, your primary task is to *****.

        Guidelines:

        
        Communication Etiquette:
        - you MUST avoid unnecessary conversation and small talk. 
        - Do NOT show appreciation in your responses, or engage in conversation. Say TERMINATE when your job of writing code is done.
        - Refrain from using appreciation phrases such as "Thank you" or "You're welcome" in your responses, if this happens say "TERMINATE" to complete the conversation.
        - After the user has run the code successfully, say "TERMINATE" to indicate that no further instructions are given and the task is complete.
        
        You will be penalized for not adhearing to the guidelines.
        """
    )
}

coding_assistant = AssistantAgent(
    name="image_classifier",
    llm_config=llm_config_mistral,
    system_message=system_messages['IMAGE_CLASSIFIER_SYSTEM_MESSAGE']
)

coding_runner = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    llm_config=llm_config_mistral,
    code_execution_config={
        "work_dir": work_dir, 
        "use_docker": False
    },
    system_message=system_messages['USER_PROXY_SYSTEM_MESSAGE']
)

In [ ]:
source_file_path = "./sample_images/"
target_folder =  "./targer_folder/"


In [ ]:
coding_runner.initiate_chat(coding_assistant, message=task)